In [1]:
from google.colab import drive
drive.mount('/content/drive')

program_location = '/content/drive/MyDrive/Colab Notebooks/PTI'

Mounted at /content/drive


---
# **0. PREPARATION**

### 0.2 Expected inputs

This script uses admin areas, settlements, and an enhanced FEWS and OECD livelihood zones shapefile.

##### ADMIN AREAS: Source file type: geopackage.
The script takes the __adm3__ layer of AdminBoundaries.gpkg. The __adm3__ layer must have a unique ID named _ADM3_CODE_, as well as codes for the two other admin sets (_ADM2_CODE_, _ADM1_CODE_).

##### SETTLEMENTS: Source file type: geopackage.
Script uses the GRID3 intermediate files produced from Auxilliary_Prep.

##### LIVELIHOOD ZONES: Source file type: geopackage (1 layer only).
The livelihood zones (LZ) used in this script are available on FEWS NET. The FEWS NET shapefiles for the four countries have already been merged together. Additionally, a new variable called _OECD_ZONE_ assigns each FEWS zone a simplified LZ alternative made by OECD and partnering orgs.
<br><br>If using a different LZ source, note that User must adapt the code to use FEWS' descriptors as the LZ variable instead of _OECD_ZONE_.

### 0.3 Outputs

This script assigns each admin area a livelihood zone (LZ): whichever LZ hosts more of the admin area's population than any other LZ. Output is a csv.

### 0.4 Pseudocode (outline of the following script)

##### Spatial join
ADM3, LZ, and GRID3 centroids to equal area projection.
<br> Spatial join ADM3 onto GRID3.
<br> Spatial join LZ onto GRID3.
<br> Convert GRID3 to dataframe (no need for geometries starting here).

##### Largest population by group
GRID3: Group by ADM3 and LZ. Sum population field (reset index to create new dataframe).
<br> NewLZ field: Select the LZ value of the datapoint which has the largest population in its ADM group.
<br> Repeat for other ADMs.

##### Aggregate to ADM
Group by ADM3. Select "first" of the NewLZ field. (or max)
<br> Repeat for other ADMs.

---
#**1. SET-UP**

In [2]:
!pip install geopandas rioxarray richdem geemap rasterio import_ipynb pyshp pycrs pyogrio rasterstats

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.7/53.7 kB 1.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 329.4/329.4 kB 9.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 20.6/20.6 MB 57.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 22.0/22.0 MB 52.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 137.9/137.9 kB 13.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 67.1 MB/s eta 0:00:00
  Created wheel for richdem: filename=richdem-0.3.4-cp310-cp310-linux_x86_64.whl size=7844536 sha256=4fee00d976a8b1b149ac7fd272b82803844460f1e7e5a3c3b11316c77353a623
  Stored in directory: /root/.cache/pip/wheels/01/61/27/5c305116b5cffecf04ed75c40f640e2baf231dae20954e31e0
  Created wheel for pycrs: filename=PyCRS-1.0.2-py3-none-any.whl size=32688 sha256=892d4a5f1ce1fd005af56d4b0e5289d60e31c998cf61c9c28b951910e7b0746c
  Stored in

In [3]:
# Built-in:
# dir(), print(), range(), format(), int(), len(), list(), max(), min(), zip(), sorted(), sum(), open(), del, = None, try except, with as, for in, if elif else
# Also: list.append(), list.insert(), list.remove(), count(), startswith(), endswith(), contains(), replace()

import os, sys, glob, re, time, subprocess, string # os.getcwd(), os.path.join(), os.listdir(), os.remove(), time.ctime(), glob.glob(), string.zfill(), string.join()
from os.path import exists # exists()
from functools import reduce # reduce()

import geopandas as gpd # read_file(), GeoDataFrame(), sjoin_nearest(), to_crs(), to_file(), .crs, buffer(), dissolve()
import pandas as pd # .dtypes, Series(), concat(), DataFrame(), read_table(), merge(), to_csv(), .loc[], head(), sample(), astype(), unique(), rename(), between(), drop(), fillna(), idxmax(), isna(), isin(), apply(), info(), sort_values(), notna(), groupby(), value_counts(), duplicated(), drop_duplicates()

from shapely.geometry import Point, LineString, Polygon, shape, MultiPoint
from shapely.ops import cascaded_union
from shapely.validation import make_valid  # in apply(make_valid)
import shapely.wkt
import matplotlib.pyplot as plt

import numpy as np # median(), mean(), tolist(), .inf
import fiona, rioxarray # fiona.open()
import rasterio # open(), write_band(), .name, .count, .width, .height. nodatavals, .meta, update(), copy(), write()

from rasterio.plot import show
from rasterio import features # features.rasterize()
from rasterio.features import shapes
from rasterio import mask # rasterio.mask.mask()
from rasterio.enums import Resampling # rasterio.enums.Resampling()
from rasterstats import zonal_stats
from osgeo import gdal, osr, ogr, gdal_array, gdalconst # Open(), SpatialReference, WarpOptions(), Warp(), GetDataTypeName(), GetRasterBand(), GetNoDataValue(), Translate(), GetProjection(), GetAttrValue()


import import_ipynb
import importlib


# Import external files
os.chdir(program_location)
!pwd
import config

sys.path.append(program_location)

import tools
importlib.reload(tools)

/content/drive/MyDrive/Colab Notebooks/PTI
importing Jupyter notebook from config.ipynb


<module 'tools' from '/content/drive/MyDrive/Colab Notebooks/PTI/tools.py'>

In [4]:
data_loc = os.path.join(os.getcwd(), 'data', config.ISO)
print(data_loc)


# The usual directories

Current_Fd = os.path.join(data_loc, 'LivelihoodZones')
Source_Fd = os.path.join(Current_Fd, 'Source')
Intermediate_Fd = os.path.join(Current_Fd, 'Intermediate')
Dest_Fd = os.path.join(Current_Fd, 'Results')


 # Generate folders if not exist:

if not os.path.exists(Source_Fd):
    os.mkdir(Source_Fd)

if not os.path.exists(Intermediate_Fd):
    os.mkdir(Intermediate_Fd)

if not os.path.exists(Dest_Fd):
    os.mkdir(Dest_Fd)


# Auxilliary sources
G3_Fd = os.path.join(data_loc, 'GRID3')
ADM_Fd = os.path.join(data_loc, 'ADM')

# Check paths
print('\n\n'.join([ Current_Fd, Source_Fd, Intermediate_Fd, Dest_Fd, G3_Fd, ADM_Fd]))

/content/drive/MyDrive/Colab Notebooks/PTI/data/GMB
/content/drive/MyDrive/Colab Notebooks/PTI/data/GMB/LivelihoodZones

/content/drive/MyDrive/Colab Notebooks/PTI/data/GMB/LivelihoodZones/Source

/content/drive/MyDrive/Colab Notebooks/PTI/data/GMB/LivelihoodZones/Intermediate

/content/drive/MyDrive/Colab Notebooks/PTI/data/GMB/LivelihoodZones/Results

/content/drive/MyDrive/Colab Notebooks/PTI/data/GMB/GRID3

/content/drive/MyDrive/Colab Notebooks/PTI/data/GMB/ADM


---
# **2. MAIN PROCESS**

## **2.1. Import SHPs**

In [5]:
# Import SHPs

# G3 Rural
G3_rural = tools. vec_import(config.RW_engine, os.path.join(G3_Fd, 'Intermediate', 'SSA_HA_pt.shp'))
G3_rural = G3_rural[['G3_ID', 'pop_un_adj', 'ADM1_CODE', 'ADM2_CODE', 'ADM3_CODE', 'geometry']]

if G3_rural.crs != 'ESRI:102022':
  G3_rural.to_crs('ESRI:102022', inplace = True)
  print('/// Convertetd G3 Rural to {}'.format(G3_rural.crs))

else:
  print('/// CRS OK')



# ADM 3
ADM3 = tools. vec_import(config.RW_engine, os.path.join(ADM_Fd, 'Source', config.original_adm3_fil))
ADM3.rename(columns = config.l_replace, inplace = True)
ADM3 = ADM3[['ADM1_CODE', 'ADM2_CODE', 'ADM3_CODE', 'geometry']]

if ADM3.crs != 'ESRI:102022':
  ADM3.to_crs('ESRI:102022', inplace = True)
  print('/// Convertetd ADM3 to {}'.format(ADM3.crs))

else:
  print('/// CRS OK')



# Livelihood Zone (or qeuivalent)

LZ = tools. vec_import(config.RW_engine, os.path.join(Source_Fd, config.original_LV_fil))
LZ = LZ[config.LZ_tar_col]

if LZ.crs != 'ESRI:102022':
  LZ.to_crs('ESRI:102022', inplace = True)
  print('/// Convertetd LZ to {}'.format(ADM3.crs))

else:
  print('CRS OK')



# Check imported data
print(G3_rural.info(), ADM3.info(), LZ.info())

Vector import complete.
GDF size:7331
PROJCS["Africa_Albers_Equal_Area_Conic",GEOGCS["WGS 84",DATUM["WGS_1984",SPHEROID["WGS 84",6378137,298.257223563,AUTHORITY["EPSG","7030"]],AUTHORITY["EPSG","6326"]],PRIMEM["Greenwich",0,AUTHORITY["EPSG","8901"]],UNIT["degree",0.0174532925199433,AUTHORITY["EPSG","9122"]],AUTHORITY["EPSG","4326"]],PROJECTION["Albers_Conic_Equal_Area"],PARAMETER["latitude_of_center",0],PARAMETER["longitude_of_center",25],PARAMETER["standard_parallel_1",20],PARAMETER["standard_parallel_2",-23],PARAMETER["false_easting",0],PARAMETER["false_northing",0],UNIT["metre",1,AUTHORITY["EPSG","9001"]],AXIS["Easting",EAST],AXIS["Northing",NORTH],AUTHORITY["ESRI","102022"]]
/// CRS OK
Vector import complete.
GDF size:120
EPSG:4326
/// Convertetd ADM3 to ESRI:102022
Vector import complete.
GDF size:6
EPSG:4326
/// Convertetd LZ to ESRI:102022
<class 'geopandas.geodataframe.GeoDataFrame'>
RangeIndex: 7331 entries, 0 to 7330
Data columns (total 6 columns):
 #   Column      Non-Null C

## **2.2 Spatial join GRID3 rural point data and Livelihood Zones**

In [6]:
G3_rural = gpd.sjoin(G3_rural, LZ, how='left', predicate='intersects')
print(G3_rural.sample(5), G3_rural.info())

<class 'geopandas.geodataframe.GeoDataFrame'>
Int64Index: 7331 entries, 0 to 7330
Data columns (total 8 columns):
 #   Column       Non-Null Count  Dtype   
---  ------       --------------  -----   
 0   G3_ID        7331 non-null   int64   
 1   pop_un_adj   7331 non-null   float64 
 2   ADM1_CODE    7255 non-null   object  
 3   ADM2_CODE    7255 non-null   object  
 4   ADM3_CODE    7255 non-null   object  
 5   geometry     7331 non-null   geometry
 6   index_right  7331 non-null   int64   
 7   NAME_ENG     7331 non-null   object  
dtypes: float64(1), geometry(1), int64(2), object(4)
memory usage: 515.5+ KB
      G3_ID  pop_un_adj ADM1_CODE  ADM2_CODE      ADM3_CODE  \
3889   3916   33.132820    GMB002  GMB002024  GMB0020240038   
3848   3875   34.584720    GMB002  GMB002024  GMB0020240037   
1086   1094   19.260771    GMB001  GMB001017  GMB0010170011   
3265   3289   78.010773    GMB005  GMB005036  GMB0050360094   
4194   4221   10.422780    GMB002  GMB002006  GMB0020060024   



In [7]:
try:
    G3_rural = G3_rural.drop(['index_right'], axis=1)
except:
    pass

## **2.3. COMPUTE THE LARGEST POPULATION BY ADM-LZ ZONE**

In [8]:
df = pd.DataFrame(G3_rural).drop(columns = 'geometry')
AllSummaries = ADM3[['ADM3_CODE', 'ADM2_CODE', 'ADM1_CODE']]

AllSummaries

,ADM3_CODE,ADM2_CODE,ADM1_CODE
0,GMB0000010000,GMB000001,GMB000
1,GMB0000010001,GMB000001,GMB000
2,GMB0000010002,GMB000001,GMB000
3,GMB0000020003,GMB000002,GMB000
4,GMB0000020004,GMB000002,GMB000
...,...,...,...
115,GMB0070200115,GMB007020,GMB007
116,GMB0070210116,GMB007021,GMB007
117,GMB0070210117,GMB007021,GMB007
118,GMB0070220118,GMB007022,GMB007


For each unique ADM3-LZ area, sum up the population of non-built-up (i.e., rural) settlements.

In [9]:
for ADM in ['ADM3', 'ADM2', 'ADM1']:

    ADM_ID = ADM + '_CODE'
    NewField = 'LZ_' + ADM

    GroupedVals = df[[ADM_ID, config.LZ_ZONE_ID, 'pop_un_adj']].groupby([ADM_ID, config.LZ_ZONE_ID], as_index=False).sum()

    print('Number of %ss with multiple livelihood zones: ' % ADM,
      len(ADM3[ADM_ID].unique()) - len(GroupedVals[ADM_ID].unique()))

    # Within each ADM group, choose the row which has the highest population.
    GroupedVals_Max = GroupedVals.loc[GroupedVals.groupby([ADM_ID])['pop_un_adj'].idxmax()]
    print(GroupedVals_Max.sample(5))

    # Join the livelihood zone with the highest rural population onto the ADM dataframe as field 'LZ'.
    AllSummaries = AllSummaries.merge(
        GroupedVals_Max.rename(columns={config.LZ_ZONE_ID : NewField})[[ADM_ID, NewField]], on=ADM_ID, how='left')

    print('Missing values: ', AllSummaries[NewField].isna().sum())
    print('Livelihood zones in %s: ' % ADM, AllSummaries[NewField].unique())


AllSummaries.sample(20)

Number of ADM3s with multiple livelihood zones:  23
        ADM3_CODE       NAME_ENG    pop_un_adj
86  GMB0070130109  Pure tropical   4858.104197
29  GMB0020240037  Pure tropical  13812.939158
39  GMB0020260050  Pure tropical  18847.256408
30  GMB0020240038  Pure tropical  15900.212894
12  GMB0010460020  Pure tropical  11359.526884
Missing values:  23
Livelihood zones in ADM3:  [nan 'Pure tropical']
Number of ADM2s with multiple livelihood zones:  3
    ADM2_CODE       NAME_ENG    pop_un_adj
33  GMB005044  Pure tropical  37050.316073
15  GMB002025  Pure tropical  10151.123238
37  GMB006037  Pure tropical  28595.475952
21  GMB003033  Pure tropical   7590.745173
10  GMB002008  Pure tropical  12469.201516
Missing values:  9
Livelihood zones in ADM2:  [nan 'Pure tropical']
Number of ADM1s with multiple livelihood zones:  0
  ADM1_CODE       NAME_ENG     pop_un_adj
2    GMB002  Pure tropical  172380.695818
4    GMB004  Pure tropical    2826.638931
6    GMB006  Pure tropical  114183.463174
5

,ADM3_CODE,ADM2_CODE,ADM1_CODE,LZ_ADM3,LZ_ADM2,LZ_ADM1
2,GMB0000010002,GMB000001,GMB000,NaN,NaN,Pure tropical
15,GMB0010380015,GMB001038,GMB001,Pure tropical,Pure tropical,Pure tropical
48,GMB0020250048,GMB002025,GMB002,Pure tropical,Pure tropical,Pure tropical
20,GMB0010460020,GMB001046,GMB001,Pure tropical,Pure tropical,Pure tropical
51,GMB0020260051,GMB002026,GMB002,Pure tropical,Pure tropical,Pure tropical
77,GMB0040410077,GMB004041,GMB004,NaN,Pure tropical,Pure tropical
18,GMB0010420018,GMB001042,GMB001,Pure tropical,Pure tropical,Pure tropical
87,GMB0050110087,GMB005011,GMB005,Pure tropical,Pure tropical,Pure tropical
42,GMB0020250042,GMB002025,GMB002,NaN,Pure tropical,Pure tropical
36,GMB0020230036,GMB002023,GMB002,Pure tropical,Pure tropical,Pure tropical


# **3. FINAL CHECK & EXPORT**

In [10]:
# Empty row check
AllSummaries.query('LZ_ADM3.isnull()', engine='python')

# ADM3 areas with no overlapping rural point (=likely urban areas) do not have LZ_ADM3 value and got 'NaN' instaed.

,ADM3_CODE,ADM2_CODE,ADM1_CODE,LZ_ADM3,LZ_ADM2,LZ_ADM1
0,GMB0000010000,GMB000001,GMB000,NaN,NaN,Pure tropical
1,GMB0000010001,GMB000001,GMB000,NaN,NaN,Pure tropical
2,GMB0000010002,GMB000001,GMB000,NaN,NaN,Pure tropical
3,GMB0000020003,GMB000002,GMB000,NaN,Pure tropical,Pure tropical
5,GMB0000020005,GMB000002,GMB000,NaN,Pure tropical,Pure tropical
6,GMB0000030006,GMB000003,GMB000,NaN,NaN,Pure tropical
7,GMB0000030007,GMB000003,GMB000,NaN,NaN,Pure tropical
8,GMB0000030008,GMB000003,GMB000,NaN,NaN,Pure tropical
40,GMB0020250040,GMB002025,GMB002,NaN,Pure tropical,Pure tropical
41,GMB0020250041,GMB002025,GMB002,NaN,Pure tropical,Pure tropical


In [11]:
# Export the result CSV to 'Result'

AllSummaries.to_csv(os.path.join(Dest_Fd, 'LivelihoodZones_ADM.csv'))